In [ ]:
import torch


x = torch.load('/home/xiangcen/RLModality/models/loss/agent_test.pt')
x[-10:]

In [1]:
from RL.Env import Env
from RL.Agent import Agent
from Training import data_spilt, ReadH5d, create_data_loader
from Training import train_seg_net, test_seg_net
from Training import test_agent
from monai.transforms import *
from monai.networks.nets import DynUNet, SwinUNETR
from monai.losses import DiceFocalLoss
import torch
import random


batch_size=6
num_epoch=1000



seg_list, rl_list, holdout_list = data_spilt('/home/xiangcen/RLModality/picai_h5', 110, 100, 10)
seg_list_promise, rl_list_promise, holdout_list_promise = data_spilt('/home/xiangcen/RLModality/promise_h5', 231, 180, 20)

train_list = rl_list + rl_list_promise
test_list = holdout_list + holdout_list_promise


eps_length = 10
learn_length = 10
batch_size = 10
n_epochs = 1


device = 'cpu'
seg_model = SwinUNETR(
    img_size = (128, 128, 32),
    in_channels = 2,
    out_channels = 1,
    depths = (2, 2, 2, 2),
    num_heads = (3, 6, 12, 24),
    drop_rate = 0.1,
    attn_drop_rate = 0.1,
    dropout_path_rate = 0.1,
    downsample="mergingv2",
    use_v2=True,
)
seg_model.load_state_dict(torch.load("/home/xiangcen/RLModality/models/segmentation.ptm", map_location=device, weights_only=True))
seg_model.eval()

agent = Agent(gamma = 0.98, alpha=0.0001, gae_lambda=0.95, policy_clip=0.2, batch_size=batch_size, n_epochs=n_epochs, device=device)
agent.load_models('/home/xiangcen/RLModality/models/rl_models/actor.ptm', '/home/xiangcen/RLModality/models/rl_models/critic.ptm')

test_dir = holdout_list[2]
env = Env(test_dir, seg_model, eps_length)


test_dice = torch.tensor(test_agent(Env(test_dir, seg_model, eps_length), agent, 10, device=device))

print(test_dice)

/home/xiangcen/miniconda3/envs/dl/lib/python3.12/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


actor and critic loaded!
tensor([[9.9346e-01, 9.9896e-01, 9.9105e-01, 9.9141e-01, 9.9969e-01, 9.9918e-01,
         3.5022e-03, 9.8890e-01, 1.8474e-08, 9.9748e-01, 9.7712e-04, 4.3139e-05,
         9.9974e-01, 9.9869e-01, 9.5884e-01, 9.9030e-01, 9.9004e-01, 9.9791e-01,
         7.4812e-09, 2.5523e-04, 9.7045e-01, 9.9997e-01, 9.9778e-01, 9.9984e-01,
         9.9053e-01, 7.4837e-02, 9.8982e-01, 9.9366e-01, 9.9582e-01, 4.3749e-05,
         9.8508e-01, 9.8250e-01]])
13
tensor([[9.9392e-01, 9.9902e-01, 9.9168e-01, 9.9174e-01, 9.9972e-01, 9.9928e-01,
         3.2066e-03, 9.8854e-01, 1.5803e-08, 9.9781e-01, 1.0071e-03, 4.5188e-05,
         9.9976e-01, 9.9881e-01, 9.6088e-01, 9.9055e-01, 9.9018e-01, 9.9808e-01,
         4.8285e-09, 2.4667e-04, 9.7051e-01, 9.9997e-01, 9.9786e-01, 9.9986e-01,
         9.9077e-01, 7.4647e-02, 9.9035e-01, 9.9390e-01, 9.9613e-01, 4.2298e-05,
         9.8518e-01, 9.8333e-01]])
27
tensor([[9.9392e-01, 9.9902e-01, 9.9168e-01, 9.9174e-01, 9.9972e-01, 9.9928e-01,
        